In [45]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import pandas as pd
import time

# Function to scrape player profile URLs from the main stats page
def get_player_profile_links():
    # Initialize the Selenium WebDriver
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service)

    # Define the URL for player stats (adjusting to the desired date range)
    players_url = "https://www.hltv.org/stats/players?startDate=2023-09-01&endDate=2024-09-01"

    # Load the page with Selenium
    driver.get(players_url)
    time.sleep(5)  # Give time for the page to fully load

    # Parse the page content with BeautifulSoup
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # Find the player stats table
    players_table = soup.find("table", class_="stats-table player-ratings-table")
    table_body = players_table.find("tbody")
    player_rows = table_body.findAll("tr")

    # List to store player profile links
    player_links = []

    # Extract relevant columns for each player
    for row in player_rows:
        cols = row.findAll("td")

        if len(cols) >= 7:  # Ensure there are at least 7 columns (the required number)
            player_name = cols[0].text.strip()  # Player Name
            profile_link = cols[0].find('a')['href']  # Player profile link

            # Full player profile link
            full_profile_link = f"https://www.hltv.org{profile_link}"

            # Add the player name and link to the list
            player_links.append({
                'Player Name': player_name,
                'Profile Link': full_profile_link
            })

    # Close the browser
    driver.quit()

    return player_links

# Function to scrape detailed stats from a player's profile page
def scrape_player_profile(player_link, player_name):
    # Initialize the Selenium WebDriver
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service)

    # Load the player's profile page
    driver.get(player_link)
    time.sleep(5)  # Give time for the page to fully load

    # Parse the page content with BeautifulSoup
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # Extract detailed stats (e.g., kills, deaths, headshot %, etc.)
    stats = {}

    try:
        stats_section = soup.find('div', class_='statistics')

        # Extracting all stats from the screenshot
        stats['Player Name'] = player_name
        stats['Total Kills'] = stats_section.find(string="Total kills").find_next().text.strip()
        stats['Headshot %'] = stats_section.find(string="Headshot %").find_next().text.strip()
        stats['Total Deaths'] = stats_section.find(string="Total deaths").find_next().text.strip()
        stats['K/D Ratio'] = stats_section.find(string="K/D Ratio").find_next().text.strip()
        stats['Damage / Round'] = stats_section.find(string="Damage / Round").find_next().text.strip()
        stats['Grenade Damage / Round'] = stats_section.find(string="Grenade dmg / Round").find_next().text.strip()
        stats['Maps Played'] = stats_section.find(string="Maps played").find_next().text.strip()
        stats['Rounds Played'] = stats_section.find(string="Rounds played").find_next().text.strip()
        stats['Kills / Round'] = stats_section.find(string="Kills / round").find_next().text.strip()
        stats['Assists / Round'] = stats_section.find(string="Assists / round").find_next().text.strip()
        stats['Deaths / Round'] = stats_section.find(string="Deaths / round").find_next().text.strip()
        stats['Saved by Teammate / Round'] = stats_section.find(string="Saved by teammate / round").find_next().text.strip()
        stats['Saved Teammates / Round'] = stats_section.find(string="Saved teammates / round").find_next().text.strip()
        stats['Rating 2.0'] = stats_section.find(string="Rating 2.0").find_next().text.strip()

    except Exception as e:
        print(f"Error extracting stats for {player_link}: {e}")

    # Close the browser
    driver.quit()

    return stats

# Function to scrape detailed stats for all players
def scrape_all_player_stats():
    player_links = get_player_profile_links()  # Get the list of player profile links

    all_player_stats = []

    for player in player_links:
        print(f"Scraping stats for {player['Player Name']} ({player['Profile Link']})")
        player_stats = scrape_player_profile(player['Profile Link'], player['Player Name'])
        all_player_stats.append(player_stats)

    # Convert the list to a DataFrame
    player_stats_df = pd.DataFrame(all_player_stats)

    return player_stats_df

# Scrape detailed player statistics for the last 12 months
detailed_player_stats_df = scrape_all_player_stats()

# Display the first few rows of the DataFrame
print(detailed_player_stats_df.head())

# Save the player statistics to a CSV file
detailed_player_stats_df.to_csv('CSGO_detailed_player_stats.csv', index=False)


Scraping stats for donk (https://www.hltv.org/stats/players/21167/donk?startDate=2023-09-01&endDate=2024-09-01)
Scraping stats for m0NESY (https://www.hltv.org/stats/players/19230/m0nesy?startDate=2023-09-01&endDate=2024-09-01)
Scraping stats for XANTARES (https://www.hltv.org/stats/players/7938/xantares?startDate=2023-09-01&endDate=2024-09-01)
Scraping stats for malbsMd (https://www.hltv.org/stats/players/11617/malbsmd?startDate=2023-09-01&endDate=2024-09-01)
Scraping stats for smooya (https://www.hltv.org/stats/players/11271/smooya?startDate=2023-09-01&endDate=2024-09-01)
Scraping stats for w0nderful (https://www.hltv.org/stats/players/20127/w0nderful?startDate=2023-09-01&endDate=2024-09-01)
Scraping stats for dav1deuS (https://www.hltv.org/stats/players/18865/dav1deus?startDate=2023-09-01&endDate=2024-09-01)
Scraping stats for deko (https://www.hltv.org/stats/players/20113/deko?startDate=2023-09-01&endDate=2024-09-01)
Scraping stats for nilo (https://www.hltv.org/stats/players/20119

Scraping stats for reiko (https://www.hltv.org/stats/players/15203/reiko?startDate=2023-09-01&endDate=2024-09-01)
Scraping stats for FL1T (https://www.hltv.org/stats/players/12732/fl1t?startDate=2023-09-01&endDate=2024-09-01)
Scraping stats for KENSI (https://www.hltv.org/stats/players/19236/kensi?startDate=2023-09-01&endDate=2024-09-01)
Scraping stats for togs (https://www.hltv.org/stats/players/20084/togs?startDate=2023-09-01&endDate=2024-09-01)
Scraping stats for d1Ledez (https://www.hltv.org/stats/players/20357/d1ledez?startDate=2023-09-01&endDate=2024-09-01)
Scraping stats for xertioN (https://www.hltv.org/stats/players/20312/xertion?startDate=2023-09-01&endDate=2024-09-01)
Scraping stats for FaNg (https://www.hltv.org/stats/players/17372/fang?startDate=2023-09-01&endDate=2024-09-01)
Scraping stats for misutaaa (https://www.hltv.org/stats/players/14176/misutaaa?startDate=2023-09-01&endDate=2024-09-01)
Scraping stats for PR (https://www.hltv.org/stats/players/22279/pr?startDate=202

Scraping stats for saffee (https://www.hltv.org/stats/players/18835/saffee?startDate=2023-09-01&endDate=2024-09-01)
Scraping stats for DSSj (https://www.hltv.org/stats/players/18447/dssj?startDate=2023-09-01&endDate=2024-09-01)
Scraping stats for s0und (https://www.hltv.org/stats/players/19817/s0und?startDate=2023-09-01&endDate=2024-09-01)
Scraping stats for khaN (https://www.hltv.org/stats/players/23317/khan?startDate=2023-09-01&endDate=2024-09-01)
Scraping stats for sdy (https://www.hltv.org/stats/players/12731/sdy?startDate=2023-09-01&endDate=2024-09-01)
Scraping stats for AZUWU (https://www.hltv.org/stats/players/22106/azuwu?startDate=2023-09-01&endDate=2024-09-01)
Scraping stats for Wicadia (https://www.hltv.org/stats/players/21243/wicadia?startDate=2023-09-01&endDate=2024-09-01)
Scraping stats for Burmylov (https://www.hltv.org/stats/players/22904/burmylov?startDate=2023-09-01&endDate=2024-09-01)
Scraping stats for juanflatroo (https://www.hltv.org/stats/players/17147/juanflatroo

Scraping stats for lauNX (https://www.hltv.org/stats/players/20761/launx?startDate=2023-09-01&endDate=2024-09-01)
Scraping stats for meyern (https://www.hltv.org/stats/players/14737/meyern?startDate=2023-09-01&endDate=2024-09-01)
Scraping stats for Lekr0 (https://www.hltv.org/stats/players/9261/lekr0?startDate=2023-09-01&endDate=2024-09-01)
Scraping stats for mopoz (https://www.hltv.org/stats/players/9254/mopoz?startDate=2023-09-01&endDate=2024-09-01)
Scraping stats for cej0t (https://www.hltv.org/stats/players/22702/cej0t?startDate=2023-09-01&endDate=2024-09-01)
Scraping stats for notineki (https://www.hltv.org/stats/players/21710/notineki?startDate=2023-09-01&endDate=2024-09-01)
Scraping stats for FpSSS (https://www.hltv.org/stats/players/20201/fpsss?startDate=2023-09-01&endDate=2024-09-01)
Scraping stats for rain (https://www.hltv.org/stats/players/8183/rain?startDate=2023-09-01&endDate=2024-09-01)
Scraping stats for susp (https://www.hltv.org/stats/players/21163/susp?startDate=2023

Scraping stats for Boombl4 (https://www.hltv.org/stats/players/11840/boombl4?startDate=2023-09-01&endDate=2024-09-01)
Scraping stats for chop (https://www.hltv.org/stats/players/19705/chop?startDate=2023-09-01&endDate=2024-09-01)
Scraping stats for KRIMZ (https://www.hltv.org/stats/players/7528/krimz?startDate=2023-09-01&endDate=2024-09-01)
Scraping stats for Tauson (https://www.hltv.org/stats/players/20301/tauson?startDate=2023-09-01&endDate=2024-09-01)
Scraping stats for Tuurtle (https://www.hltv.org/stats/players/14394/tuurtle?startDate=2023-09-01&endDate=2024-09-01)
Scraping stats for b4rtiN (https://www.hltv.org/stats/players/17585/b4rtin?startDate=2023-09-01&endDate=2024-09-01)
Scraping stats for MistR (https://www.hltv.org/stats/players/21199/mistr?startDate=2023-09-01&endDate=2024-09-01)
Scraping stats for Queenix (https://www.hltv.org/stats/players/13749/queenix?startDate=2023-09-01&endDate=2024-09-01)
Scraping stats for bobeksde (https://www.hltv.org/stats/players/21159/bobek

Scraping stats for X5G7V (https://www.hltv.org/stats/players/19575/x5g7v?startDate=2023-09-01&endDate=2024-09-01)
Scraping stats for Snax (https://www.hltv.org/stats/players/2553/snax?startDate=2023-09-01&endDate=2024-09-01)
Scraping stats for The eLiVe (https://www.hltv.org/stats/players/14388/the-elive?startDate=2023-09-01&endDate=2024-09-01)
Scraping stats for Patti (https://www.hltv.org/stats/players/13077/patti?startDate=2023-09-01&endDate=2024-09-01)
Scraping stats for virtuoso (https://www.hltv.org/stats/players/22476/virtuoso?startDate=2023-09-01&endDate=2024-09-01)
Scraping stats for STYKO (https://www.hltv.org/stats/players/6904/styko?startDate=2023-09-01&endDate=2024-09-01)
Scraping stats for naz (https://www.hltv.org/stats/players/21155/naz?startDate=2023-09-01&endDate=2024-09-01)
Scraping stats for zmb (https://www.hltv.org/stats/players/14010/zmb?startDate=2023-09-01&endDate=2024-09-01)
Scraping stats for amster (https://www.hltv.org/stats/players/19617/amster?startDate=2